**<h3>Traditional Model for sentiment analysis FYP</h3>**



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC

# Example DataFrame (replace this with your actual dataset)
# df = pd.read_csv('your_dataset.csv')  # Load your dataset here if using an actual file
data = pd.read_csv('C:/Users/airul hafiq/Desktop/DataPreprocessing/DataPreprocessing/Model/fb+reddit.csv')
df = pd.DataFrame(data)


# # Count the occurrences of each sentiment
# sentiment_counts = df['Sentiment'].value_counts().reset_index()
# sentiment_counts.columns = ['sentiment', 'Count']

# # Plotting
# sns.barplot(x='sentiment', y='Count', data=sentiment_counts, palette='viridis')
# plt.title('Sentiment Count')
# plt.xlabel('Sentiment')
# plt.ylabel('Count')
# plt.show()
df.info()


In [ ]:
df['Sentiment'].value_counts()

In [ ]:
# # Example DataFrame (replace this with your actual dataset)
# # df = pd.read_csv('your_dataset.csv')  # Load your dataset here if using an actual file
# data2 =pd.read_csv('C:/Users/airul hafiq/Desktop/DataPreprocessing/DataPreprocessing/merged/fb+reddit.csv')
# df2 = pd.DataFrame(data2)


# # Count the occurrences of each sentiment
# sentiment_counts2 = df2['Sentiment'].value_counts().reset_index()
# sentiment_counts2.columns = ['sentiment1', 'Count1']

# # Plotting
# sns.barplot(x='sentiment1', y='Count1', data=sentiment_counts2, palette='viridis')
# plt.title('Sentiment Count')
# plt.xlabel('Sentiment')
# plt.ylabel('Count')
# plt.show()

In [ ]:
# data2['Sentiment'].value_counts()

In [ ]:
if df.empty:
    print('DataFrame is empty!')

In [ ]:
data = pd.read_csv('C:/Users/airul hafiq/Desktop/DataPreprocessing/DataPreprocessing/Model/fb+reddit.csv')
df = pd.DataFrame(data)
# 2. Split the data into features (X) and target (y)
X = df['comment']  # Text data (features)
y = df['Sentiment']  # Sentiment labels (target)

In [ ]:
# 3. Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
#Convert text data to numerical data using CountVectorizer (Bag-of-Words model)
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
param_combinations = [
    {'C': 1, 'kernel': 'linear'},
    {'C': 1, 'kernel': 'rbf'},
    {'C': 1, 'kernel': 'sigmoid'},
    {'C': 10, 'kernel': 'linear'},
    {'C': 10, 'kernel': 'rbf'},
    {'C': 10, 'kernel': 'sigmoid'},
    {'C': 100, 'kernel': 'linear'},
    {'C': 100, 'kernel': 'rbf'},
    {'C': 100, 'kernel': 'sigmoid'},
]

# Store results
results = []


# Iterate through parameter combinations and evaluate the model
for params in param_combinations:
    svm_model = SVC(C=params['C'], 
                   kernel=params['kernel'], 
                   random_state=42)
    svm_model.fit(X_train_tfidf, y_train)  
    
    # Predict on the training set (optional)
    y_train_pred = svm_model.predict(X_train_tfidf)

    # Make predictions
    y_pred =svm_model.predict(X_test_tfidf)
    
    # Calculate accuracy
    training_accuracy = accuracy_score(y_train, y_train_pred)
    testing_accuracy = accuracy_score(y_test, y_pred)
    
    # Append results
    results.append({
        'C': params['C'],
        'kernel': params['kernel'],
        'training_accuracy': f"{training_accuracy * 100 :.2f}%" ,
        'testing_accuracy': f"{testing_accuracy * 100 :.2f}%"
    })
    
    # Print classification report for each configuration
    print(f"Parameters: C={params['C']}, kernel={params['kernel']}")

    # Print evaluation metrics for training  on each configuration
    print("Training Accuracy:", f"{accuracy_score(y_train, y_train_pred) * 100:.2f}%")
    print("\nClassification Report:\n", classification_report(y_train, y_train_pred))
    
    print(f"Accuracy: {testing_accuracy * 100:.2f}%")
    print(classification_report(y_test, y_pred))
    print("-" * 50)

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)

# Display the results
print("Summary of Results:")
print(results_df)

In [ ]:
# Plot training vs. testing accuracy
plt.figure(figsize=(12, 6))

# Iterate over kernel types to visualize trends
for kernel in results_df['kernel'].unique():
    kernel_subset = results_df[results_df['kernel'] == kernel]
    plt.plot(kernel_subset['C'], kernel_subset['training_accuracy'], label=f"Train (Kernel={kernel})", marker='o')
    plt.plot(kernel_subset['C'], kernel_subset['testing_accuracy'], label=f"Test (Kernel={kernel})", linestyle='--', marker='x')

plt.title('SVM Overfitting and Underfitting Analysis')
plt.xlabel('C (Regularization Parameter)')
plt.ylabel('Accuracy')
plt.xscale('log')  # Scale C axis logarithmically for better visualization
plt.legend()
plt.grid()
plt.show()

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.svm import SVC
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('C:/Users/airul hafiq/Desktop/DataPreprocessing/DataPreprocessing/Model/fb+reddit.csv')
df = pd.DataFrame(data)
# 2. Split the data into features (X) and target (y)
X = df['comment']  # Text data (features)
y = df['Sentiment']  # Sentiment labels (target)

# 3. Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Define the SVM model (adjust parameters as needed)
svm_model = SVC(C=1, kernel='linear', random_state=42)

# Compute the learning curve
train_sizes, train_scores, test_scores = learning_curve(
    svm_model, 
    X_train_tfidf,  # Feature matrix
    y,        # Target labels
    cv=5,     # Number of cross-validation folds
    train_sizes=np.linspace(0.1, 1.0, 10),  # Proportion of the dataset to use for training
    scoring='accuracy',  # Evaluation metric
    random_state=42
)

# Calculate mean and standard deviation for training and validation scores
train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
test_mean = test_scores.mean(axis=1)
test_std = test_scores.std(axis=1)

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training Accuracy', color='blue', marker='o')
plt.plot(train_sizes, test_mean, label='Validation Accuracy', color='green', marker='o')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.2)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color='green', alpha=0.2)

plt.title('Learning Curve for SVM')
plt.xlabel('Training Size')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.grid()
plt.show()


In [ ]:
from sklearn.model_selection import learning_curve, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load data
data = pd.read_csv('C:/Users/airul hafiq/Desktop/DataPreprocessing/DataPreprocessing/Model/(bert)fb+reddit.csv')
df = pd.DataFrame(data)

# Split the data into features (X) and target (y)
X = df['comment']  # Text data (features)
y = df['Sentiment']  # Sentiment labels (target)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Transform text data to numerical data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Combine X_train_tfidf and y_train for learning_curve
X_train_full_tfidf = vectorizer.fit_transform(X)  # Use the full dataset for learning_curve

# Define the SVM model
svm_model = SVC(C=1, kernel='linear', random_state=42)

# Compute the learning curve
train_sizes, train_scores, test_scores = learning_curve(
    svm_model, 
    X_train_full_tfidf,  # Full transformed feature matrix
    y,                   # Full target labels
    cv=5,                # Number of cross-validation folds
    train_sizes=np.linspace(0.1, 1.0, 10),  # Proportion of the dataset to use for training
    scoring='accuracy',  # Evaluation metric
)

# Calculate mean and standard deviation for training and validation scores
train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
test_mean = test_scores.mean(axis=1)
test_std = test_scores.std(axis=1)

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training Accuracy', color='blue', marker='o')
plt.plot(train_sizes, test_mean, label='Validation Accuracy', color='green', marker='o')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.2)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color='green', alpha=0.2)

plt.title('Learning Curve for SVM')
plt.xlabel('Training Size')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.grid()
plt.show()


In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import numpy as np

train_sizes, train_scores, test_scores = learning_curve(
    svm_model, X, y, cv=5, train_sizes=np.linspace(0.1, 1.0, 10), scoring='accuracy')

# Mean and Std Dev
train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
test_mean = test_scores.mean(axis=1)
test_std = test_scores.std(axis=1)

# Plot
plt.plot(train_sizes, train_mean, label='Training Accuracy', color='blue')
plt.plot(train_sizes, test_mean, label='Validation Accuracy', color='green')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.2)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color='green', alpha=0.2)

plt.title('Learning Curve')
plt.xlabel('Training Size')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()


In [ ]:
# 9. Create confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)


In [ ]:
# 10. Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=svm_model.classes_, yticklabels=svm_model.classes_)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Example of predicting sentiment for new text
new_text = ["This product is really amazing!", "ronaldo is the best player in the world", "I hate this product","robot is the best movie ever"]
new_text_tfidf = vectorizer.transform(new_text)
predictions = svm_model.predict(new_text_tfidf)
print("\nPredictions for new text:", predictions)

TESTING SVM

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

# Load the dataset (replace with your actual dataset path)
df = pd.read_csv('C:/Users/airul hafiq/Desktop/DataPreprocessing/DataPreprocessing/Model/(bert)fb+reddit.csv')

# Assuming the dataset has 'comment' and 'Sentiment' columns
X = df['comment']  # Text data
y = df['Sentiment']  # Sentiment labels

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()#(ngram_range=(1, 2))  # Using n-grams for richer features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Define an SVM model with specific parameters
svm_model = SVC(C=1, kernel='linear', random_state=42)

# Cross-Validation
cv_scores = cross_val_score(svm_model, X_train_tfidf, y_train, cv=5)
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Mean CV Accuracy: {cv_scores.mean():.2f}")

# Fit the model and calculate accuracies
svm_model.fit(X_train_tfidf, y_train)
train_accuracy = svm_model.score(X_train_tfidf, y_train)
test_accuracy = svm_model.score(X_test_tfidf, y_test)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")

# Generate Learning Curves
train_sizes, train_scores, validation_scores = learning_curve(
    svm_model, X_train_tfidf, y_train, cv=5, train_sizes=np.linspace(0.1, 1.0, 10), scoring='accuracy'
)

# Calculate mean and standard deviation
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
validation_scores_mean = np.mean(validation_scores, axis=1)
validation_scores_std = np.std(validation_scores, axis=1)

# Plot Learning Curves
plt.figure(figsize=(12, 6))
plt.plot(train_sizes, train_scores_mean, 'o-', label="Training Score", color="blue")
plt.plot(train_sizes, validation_scores_mean, 'o-', label="Validation Score", color="orange")
plt.fill_between(
    train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="blue"
)
plt.fill_between(
    train_sizes, validation_scores_mean - validation_scores_std, validation_scores_mean + validation_scores_std, alpha=0.1, color="orange"
)

plt.title("Learning Curve for SVM (C=10, Kernel=rbf)")
plt.xlabel("Training Set Size")
plt.ylabel("Accuracy")
plt.legend(loc="best")
plt.grid()
plt.show()


<h3>Random Forest</h3>

In [ ]:
# # Import necessary libraries
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics import accuracy_score, classification_report

# # Load the dataset (replace with your actual dataset path)
# df = pd.read_csv('C:/Users/airul hafiq/Desktop/DataPreprocessing/DataPreprocessing/Model/(bert)fb+reddit.csv')

# # Assuming the dataset has 'comment' and 'Sentiment' columns
# X = df['comment']  # Text data
# y = df['Sentiment']  # Sentiment labels

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# # TF-IDF Vectorization
# vectorizer = TfidfVectorizer(max_features=5000)  # Limit features to prevent memory issues
# X_train_tfidf = vectorizer.fit_transform(X_train)  # Transform training data
# X_test_tfidf = vectorizer.transform(X_test)  # Transform testing data

# # Parameter combinations for Random Forest
# param_combinations = [
#     {'n_estimators': 100, 'max_depth': 10},
#     {'n_estimators': 100, 'max_depth': 20},
#     {'n_estimators': 100, 'max_depth': None},
#     {'n_estimators': 200, 'max_depth': 10},
#     {'n_estimators': 200, 'max_depth': 20},
#     {'n_estimators': 200, 'max_depth': None},
#     {'n_estimators': 500, 'max_depth': 10},
#     {'n_estimators': 500, 'max_depth': 20},
# ]

# # Store results
# results = []




# # Iterate through parameter combinations and evaluate the model
# for params in param_combinations:
#     rf_model = RandomForestClassifier(n_estimators=params['n_estimators'], 
#                                       max_depth=params['max_depth'], 
#                                       random_state=42)
#     rf_model.fit(X_train_tfidf, y_train)  # Use transformed numerical data
    
#     # Predict on the training set (optional)
#     y_train_pred = rf_model.predict(X_train_tfidf)

    

#     # Make predictions
#     y_pred = rf_model.predict(X_test_tfidf)
    
#     # Calculate accuracy
#     training_accuracy = accuracy_score(y_train, y_train_pred)
#     testing_accuracy = accuracy_score(y_test, y_pred)
    
#     # Append results
#     results.append({
#         'n_estimators': params['n_estimators'],
#         'max_depth': params['max_depth'],
#         'training_accuracy': f"{training_accuracy * 100 :.2f}%" ,
#         'testing_accuracy': f"{testing_accuracy * 100 :.2f}%"
#     })
    
#     # Print classification report for each configuration
#     print(f"Parameters: n_estimators={params['n_estimators']}, max_depth={params['max_depth']}")

#     # Print evaluation metrics for training  on each configuration
#     print("Training Accuracy:", f"{accuracy_score(y_train, y_train_pred) * 100:.2f}%")
#     print("\nClassification Report:\n", classification_report(y_train, y_train_pred))
    
#     print(f"Accuracy: {testing_accuracy * 100:.2f}%")
#     print(classification_report(y_test, y_pred))
#     print("-" * 50)

# # Convert results to a DataFrame for better visualization
# results_df = pd.DataFrame(results)

# # Display the results
# print("Summary of Results:")
# print(results_df)


# # Plot train vs test accuracy to visualize overfitting
# # import matplotlib.pyplot as plt

# # plt.figure(figsize=(12, 6))
# # for n_estimate in set(results_df['n_estimators']):
# #     subset = results_df[results_df['n_estimators'] == n_estimate]
# #     plt.plot(subset['max_depth'], subset['training_accuracy'], label=f'Train Acc (n_estimate={n_estimate})', marker='o')
# #     plt.plot(subset['max_depth'], subset['testing_accuracy'], label=f'Test Acc (n_estimate={n_estimate})', linestyle='--', marker='x')

# # plt.title('Random Forest Experiment Results: Train vs Test Accuracy')
# # plt.xlabel('Max Depth ( Parameter)')
# # plt.ylabel('Accuracy (%)')
# # plt.legend()
# # plt.grid()
# # plt.show()


In [ ]:
# from sklearn.metrics import  confusion_matrix
# import matplotlib.pyplot as plt
# import seaborn as sns

# # 9. Create confusion matrix
# conf_matrix = confusion_matrix(y_test, y_pred)

# # 10. Plot confusion matrix
# plt.figure(figsize=(10, 7))
# sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=rf_model.classes_, yticklabels=rf_model.classes_)
# plt.ylabel('Actual')
# plt.xlabel('Predicted')
# plt.title('Confusion Matrix')
# plt.show()


<h3>Naive Bayes</h3>

To check if a model is overfitting or underfitting, compare its performance on the training data to its performance on a separate validation or test set; if the model performs significantly better on the training data than on the validation/test data, it is likely overfitting, while if it performs poorly on both sets, it is likely underfitting

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset (replace with your actual dataset path)
df = pd.read_csv('C:/Users/airul hafiq/Desktop/DataPreprocessing/DataPreprocessing/Model/fb+reddit.csv')

# Assuming the dataset has 'comment' and 'Sentiment' columns
X = df['comment']  # Text data
y = df['Sentiment']  # Sentiment labels

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Parameter combinations for experiments
experiments = [
    {"alpha": 0.1,"ngram_range": (1, 1)},
    {"alpha": 0.1, "ngram_range": (1, 2)},
    {"alpha": 0.5, "ngram_range": (1, 1)},
    {"alpha": 0.5, "ngram_range": (1, 2)},
    {"alpha": 1.0, "ngram_range": (1, 1)},
    {"alpha": 1.0,"ngram_range": (1, 2)},
    {"alpha": 2.0, "ngram_range": (1, 1)},
    {"alpha": 2.0,"ngram_range": (1, 2)},
]

# Store results
results = []

# Iterate through parameter combinations and evaluate the model
for exp in experiments:
    print(f"Experiment: alpha={exp['alpha']}, ngram_range={exp['ngram_range']}")

    # TF-IDF Vectorization with specified parameters
    vectorizer = TfidfVectorizer(ngram_range=exp['ngram_range'])
    X_train_tfidf = vectorizer.fit_transform(X_train)  # Transform training data
    X_test_tfidf = vectorizer.transform(X_test)  # Transform testing data

    # Initialize and train Naive Bayes model
    nb_model = MultinomialNB(alpha=exp['alpha'])
    nb_model.fit(X_train_tfidf, y_train)  # Fit the model on training data

    # Predict on training and testing sets
    y_train_pred = nb_model.predict(X_train_tfidf)
    y_test_pred = nb_model.predict(X_test_tfidf)

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    # Append results
    results.append({
        'alpha': exp['alpha'],
        'ngram_range': exp['ngram_range'],
        'train_accuracy': f"{test_accuracy * 100:.2f}%",
        'test_accuracy': f"{test_accuracy * 100:.2f}%" 
    })

    # Print classification reports
    print("Training Classification Report:\n", classification_report(y_train, y_train_pred))
    print(f"Testing Accuracy: {test_accuracy * 100:.2f}% ")
    print("Testing Classification Report:\n", classification_report(y_test, y_test_pred))
    print("-" * 50)

# Convert results to DataFrame for visualization
results_df = pd.DataFrame(results)

# Display results
print("Experiment Results:")
print(results_df)

# Plot train vs test accuracy to visualize overfitting
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
for ngram in set(results_df['ngram_range']):
    subset = results_df[results_df['ngram_range'] == ngram]
    plt.plot(subset['alpha'], subset['train_accuracy'], label=f'Train Acc (ngram={ngram})', marker='o')
    plt.plot(subset['alpha'], subset['test_accuracy'], label=f'Test Acc (ngram={ngram})', linestyle='--', marker='x')

plt.title('Naive Bayes Experiment Results: Train vs Test Accuracy')
plt.xlabel('Alpha (Smoothing Parameter)')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid()
plt.show()


In [ ]:
from sklearn.metrics import  confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# 9. Create confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# 10. Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=nb_model.classes_, yticklabels=nb_model.classes_)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()
